# Bestemmelse af fraktilen for hvert rum styret af det termiske indeklima 


In [5]:
from eppy import modeleditor
from eppy.modeleditor import IDF
import eppy

iddfile = "/Users/sararhiger/Desktop/V9-6-0-Energy+.idd"
IDF.setiddname(iddfile)
idfname = "/Users/sararhiger/Desktop/Metode3.idf"
epwfile = "/Users/sararhiger/Desktop/Bachelor-project/DRY_2001-2010_epw_fikset.epw"

In [6]:
import json

with open('eplusout_hourly_flow_performance.json') as file: 
    data_flow = json.load(file)

#### Funktion der tjekker overophedningstimerne i hverdage fra 8-17 og sommerferien (28/6-8/8)

In [7]:
import numpy as np
from datetime import datetime

def check_max_temp(data, max_temp, start_tid, slut_tid, årstal):
    værelser = np.array([0]*84)

    for row in data['Rows']:
        key, value = list(row.items())[0]
        hour = int(key[6:8])
        month = int(key[0:2])
        day = int(key[3:5])

        if month==7 or (month==6 and day>=28) or (month==8 and day<=8):
            pass
        
        elif start_tid <= hour <= slut_tid:
            dato = datetime.strptime(key, '%m/%d %H:%M:%S').replace(year=årstal)
            if dato.weekday() < 5:  # 5 og 6 weekday er weekender
                # np.array(value) > max_temp er True eller False værdier (1 eller 0)
                værelser += np.array(value) > max_temp
                
    return værelser

In [9]:
import json
import numpy as np
idf = IDF(idfname, epwfile)
idf1 = IDF(idfname)
idf.run(ep_version="9-6-0")

with open("eplusout_hourly.json") as f:
    data_temp = json.load(f)

overtemp = check_max_temp(data_temp, max_temp=26, start_tid=8, slut_tid=17, årstal=2010)
print(overtemp)

## Liste med fraktiler som start er alle 100%
k = [100]*84

for _ in range(50):

    flow = []

    for i in range(84):
        ## Hvis overophedningstimerne er mindre end 100 timer bliver fraktilen 1% mindre
        if overtemp[i] < 100:
            k[i]-=1

        ## Tom liste som skal indeholde alle 8760 flows pr rum
        flow_hourly=[]

        ## Lav liste med alle luftmængder for et rum 
        for j in range(len(data_flow['Rows'])): 
            flow_hourly.append(list(data_flow['Rows'][j].values())[0][i])
        
        ## Sorter rummets flows
        flow_sorted = sorted(flow_hourly)

        ## Find den k'e værdi og append til flow som er den dimensionerende luftmængde 
        flow.append(flow_sorted[round(8760/100*k[i]-1)])

    ## Opdater IDF'en med de nye flows 
    for i in range(len(idf1.idfobjects['HVACTemplate:Zone:IdealLoadsAirSystem'])):
        idf1.idfobjects['HVACTemplate:Zone:IdealLoadsAirSystem'][i]["Outdoor_Air_Flow_Rate_per_Zone"] = flow[i]

    idf1.save()

    idfname = "/Users/sararhiger/Desktop/Metode3.idf"
    idf = IDF(idfname, epwfile)
    idf1 = IDF(idfname)
    idf.run(ep_version="9-6-0")

    with open("eplusout_hourly.json") as f:
        data = json.load(f)

    overtemp = check_max_temp(data, max_temp=26, start_tid=8, slut_tid=17, årstal=2010)
    print(overtemp)
    print(k)






/Applications/EnergyPlus-9-6-0/energyplus --weather /Users/sararhiger/Desktop/Bachelor-project/DRY_2001-2010_epw_fikset.epw --output-directory /Users/sararhiger/Desktop/Bachelor-project/Kode --idd /Users/sararhiger/Desktop/V9-6-0-Energy+.idd --expandobjects /Users/sararhiger/Desktop/Bachelor-project/Kode/in.idf

[ 0  0  0  0  0  0  0  5  4  0  0  6  4  3  0  0  0  0  0  0  5  8  0  0
  0  0  0  0  0  0  0  0  0  0  0  6  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0 10  8  8  8  8  9  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0]

/Applications/EnergyPlus-9-6-0/energyplus --weather /Users/sararhiger/Desktop/Bachelor-project/DRY_2001-2010_epw_fikset.epw --output-directory /Users/sararhiger/Desktop/Bachelor-project/Kode --idd /Users/sararhiger/Desktop/V9-6-0-Energy+.idd --expandobjects /Users/sararhiger/Desktop/Bachelor-project/Kode/in.idf

[ 0  0  0  0  0  0  0 26 52  0  1 11 11 11  0  0  0  0  0  0 19 22  0  0
  0  0  0  0  0  0  0  0  0  0  0 12  0 

In [31]:
with open("eplusout_hourly.json") as f:
        data_temp = json.load(f)

overtemp = check_max_temp(data_temp, max_temp=26, start_tid=8, slut_tid=17, årstal=2010)

for i in range(84):
    if overtemp[i]>100:
        k[i]+=1

In [34]:
k = [59, 56, 77, 58, 56, 56, 50, 79, 88, 63, 63, 81, 75, 76, 50, 50, 50, 50, 54, 56, 71, 77, 54, 55, 65, 60, 59, 59, 52, 51, 50, 54, 50, 50, 50, 74, 60, 54, 60, 58, 50, 56, 59, 57, 55, 61, 50, 50, 50, 56, 54, 57, 57, 57, 57, 57, 56, 50, 50, 54, 68, 81, 80, 80, 79, 81, 64, 60, 61, 61, 52, 52, 50, 50, 50, 50, 50, 52, 65, 50, 51, 61, 67, 50]
flow = []

for i in range(84):
    ## Tom liste som skal indeholde alle 8760 flows pr rum
    flow_hourly=[]

    ## Lav liste med alle luftmængder for et rum 
    for j in range(len(data_flow['Rows'])): 
        flow_hourly.append(list(data_flow['Rows'][j].values())[0][i])
    
    ## Sorter rummets flows
    flow_sorted = sorted(flow_hourly)

    ## Find den k'e værdi og append til flow som er den dimensionerende luftmængde 
    flow.append(flow_sorted[round(8760/100*k[i]-1)])

iddfile = "/Users/sararhiger/Desktop/V9-6-0-Energy+.idd"
fname1 = "/Users/sararhiger/Desktop/Metode3.idf"
IDF.setiddname(iddfile)
idf1 = IDF(fname1)

for i in range(len(idf1.idfobjects['HVACTemplate:Zone:IdealLoadsAirSystem'])):
        idf1.idfobjects['HVACTemplate:Zone:IdealLoadsAirSystem'][i]["Outdoor_Air_Flow_Rate_per_Zone"] = flow[i]

idf1.save()

print(k)
print(flow)

[59, 56, 77, 58, 56, 56, 50, 79, 88, 63, 63, 81, 75, 76, 50, 50, 50, 50, 54, 56, 71, 77, 54, 55, 65, 60, 59, 59, 52, 51, 50, 54, 50, 50, 50, 74, 60, 54, 60, 58, 50, 56, 59, 57, 55, 61, 50, 50, 50, 56, 54, 57, 57, 57, 57, 57, 56, 50, 50, 54, 68, 81, 80, 80, 79, 81, 64, 60, 61, 61, 52, 52, 50, 50, 50, 50, 50, 52, 65, 50, 51, 61, 67, 50]
[0.2005084188006853, 0.010686949524531924, 0.00719976644041184, 0.0034928674089211545, 0.0003148337250799229, 0.00033162495714310315, 0.0002645958884991627, 0.059397273959447175, 0.06343653037367313, 0.09145955611305685, 0.09418910653604737, 0.31983645298506636, 0.262741082892701, 0.26395006429718915, 0.0002602819984818427, 0.00029081584235646305, 5.6082024863019984e-05, 0.0003879029945990598, 0.06572127542150255, 0.07553011110542468, 0.0391258292843475, 0.04817680448730361, 0.05966147741636841, 0.06243585978847502, 0.003756746298923383, 0.0036993312915430203, 0.0006014583630901788, 0.0003626982981811352, 0.00028185641478865214, 0.00015753330589166897, 0.

## Læg 1 procent til fraktilen ind til alle timer er under 1000 ppm 

In [36]:
import json
import numpy as np
idfname = "/Users/sararhiger/Desktop/Metode3_fractiles.idf"
idf = IDF(idfname, epwfile)
idf1 = IDF(idfname)
idf.run(ep_version="9-6-0")

with open("eplusout_hourly.json") as f:
    data_CO2 = json.load(f)

overtemp = check_max_temp(data_CO2, max_temp=1000, start_tid=8, slut_tid=17, årstal=2010)
print(overtemp)

## Liste med fraktiler hvor overophendingtimer er overholdt
k = [59, 56, 77, 58, 56, 56, 50, 79, 88, 63, 63, 81, 75, 76, 50, 50, 50, 50, 54, 56, 71, 77, 54, 55, 65, 60, 59, 59, 52, 51, 50, 54, 50, 50, 50, 74, 60, 54, 60, 58, 50, 56, 59, 57, 55, 61, 50, 50, 50, 56, 54, 57, 57, 57, 57, 57, 56, 50, 50, 54, 68, 81, 80, 80, 79, 81, 64, 60, 61, 61, 52, 52, 50, 50, 50, 50, 50, 52, 65, 50, 51, 61, 67, 50]

for _ in range(20):

    flow = []

    for i in range(84):
        ## Hvis der er timer hvor der er over 1000 ppm lægges 1% til fraktilen
        if overtemp[i] > 0:
            k[i]+=1

        ## Tom liste som skal indeholde alle 8760 flows pr rum
        flow_hourly=[]

        ## Lav liste med alle luftmængder for et rum 
        for j in range(len(data_flow['Rows'])): 
            flow_hourly.append(list(data_flow['Rows'][j].values())[0][i])
        
        ## Sorter rummets flows
        flow_sorted = sorted(flow_hourly)

        ## Find den k'e værdi og append til flow som er den dimensionerende luftmængde 
        flow.append(flow_sorted[round(8760/100*k[i]-1)])

    ## Opdater IDF'en med de nye flows 
    for i in range(len(idf1.idfobjects['HVACTemplate:Zone:IdealLoadsAirSystem'])):
        idf1.idfobjects['HVACTemplate:Zone:IdealLoadsAirSystem'][i]["Outdoor_Air_Flow_Rate_per_Zone"] = flow[i]

    idf1.save()

    idfname = "/Users/sararhiger/Desktop/Metode3_fractiles.idf"
    idf = IDF(idfname, epwfile)
    idf1 = IDF(idfname)
    idf.run(ep_version="9-6-0")

    with open("eplusout_hourly.json") as f:
        data_CO2 = json.load(f)

    overtemp = check_max_temp(data_CO2, max_temp=1000, start_tid=8, slut_tid=17, årstal=2010)
    print(overtemp)
    print(k)


/Applications/EnergyPlus-9-6-0/energyplus --weather /Users/sararhiger/Desktop/Bachelor-project/DRY_2001-2010_epw_fikset.epw --output-directory /Users/sararhiger/Desktop/Bachelor-project/Kode --idd /Users/sararhiger/Desktop/V9-6-0-Energy+.idd --expandobjects /Users/sararhiger/Desktop/Bachelor-project/Kode/in.idf

[ 315   13    0    0    0    0    0    0    0  866  820    0    0    0
    0    0    0    0  944  892    0    0 1004  971    0    0    0    0
    0    0    0    0    0    0    0    0    0    0  875  899    0  967
  899  959  947    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
  490  467    0    0    0    0    0    0    0    0    0    0    0    0]

/Applications/EnergyPlus-9-6-0/energyplus --weather /Users/sararhiger/Desktop/Bachelor-project/DRY_2001-2010_epw_fikset.epw --output-directory /Users/sararhiger/Desktop/Bachelor-project/Kode --idd /Users/sararhiger/Desktop/V9-6-0-Energy+.idd --expandobjects

KeyboardInterrupt: 

In [1]:
Endelige_fraktiler = [68, 59, 77, 58, 56, 56, 50, 79, 88, 68, 68, 81, 75, 76, 50, 50, 50, 50, 66, 66, 71, 77, 67, 67, 65, 60, 59, 59, 52, 51, 50, 54, 50, 50, 50, 74, 60, 54, 68, 68, 50, 67, 68, 68, 67, 61, 50, 50, 50, 56, 54, 57, 57, 57, 57, 57, 56, 50, 50, 54, 68, 81, 80, 80, 79, 81, 64, 60, 61, 61, 62, 62, 50, 50, 50, 50, 50, 52, 65, 50, 51, 61, 67, 50]

In [2]:
Endelige_fraktiler[41]

67